In [1]:
!pip install rioxarray
!pip install geopandas
!pip install tqdm
!pip install google.cloud
!pip install configparser
!pip install optuna
!pip install catboost
!pip install pickle5
!pip install shap
!pip install geocube
!pip install fsspec
!pip install gcsfs

     |████████████████████████████████| 46 kB 1.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.3 MB 25.2 MB/s 
     |████████████████████████████████| 6.3 MB 27.9 MB/s 
  Created wheel for rioxarray: filename=rioxarray-0.7.1-py3-none-any.whl size=54118 sha256=a7cd438805277d097e235982596775c61a782ce2424e8a8547dfa8ccce6f25f4
  Stored in directory: /root/.cache/pip/wheels/42/9b/1c/acf2904f7ee44be3c0b8ffc7ea4fafebb135aee00242bef86f
Successfully built rioxarray
     |████████████████████████████████| 1.0 MB 4.7 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 308 kB 2.1 MB/s 
     |████████████████████████████████| 80 kB 7.8 MB/s 
     |████████████████████████████████| 209 kB 25.3 MB/s 
     |████████████████████████████████| 75 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 28.4 MB/s 
     |████

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/03 PEMETAAN/01 MODEL - DRIVE

import os
os.listdir()

/content/drive/MyDrive/03 PEMETAAN/01 MODEL - DRIVE


['kerangka-spasial',
 'config',
 '__pycache__',
 'catboost_info',
 '02_Exploration Data.ipynb',
 '03_Outlier Removal.ipynb',
 'bps_peta_processing.py',
 'temp_raster',
 '06_Processing.ipynb',
 '04_Feature Elimination and Tuning Hyper Parameter.ipynb',
 'model_3001.pickle',
 '05_Modeling and Explaining Model.ipynb',
 'bps_peta_download_citra.py',
 '00_Download Citra GEE-GDrive.ipynb',
 '01_Creating Training Sample from Vector and Grid.ipynb']

In [4]:
import rioxarray as rio
import geopandas as gpd
import pandas as pd
from tqdm.notebook import tqdm_notebook
import configparser
from bps_peta_processing import iterate_raster, preprocessing_tif_vector,iterate_geom_

config = configparser.ConfigParser()
config.read('config/config_ecoregion_3001.ini')
folder=config['default']['folder']
v_name=config['input']['vector']
train_data=config['output']['train_data']
list_imagery=config['input']['raster_train'].split(',')

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.


## Load Vector training data

Before run the cell please make sure that the data is in GPKG format to ease reading. 

In [5]:
train_vector_path='kerangka-spasial/vector-training/'+v_name
train_vector=gpd.read_file(train_vector_path)

In [6]:
train_vector['id'].unique()

array([12,  7, 11,  9,  5,  1,  4, 33,  2,  6, 31, 10])

## Load Raster training data

In [7]:
data_=iterate_raster(folder, list_imagery,train_vector)

Creating training data for:  setahun_ID-1614_sentinel2_10m_med_mean_rev.tif


Processing clipping raster with vector:   0%|          | 0/145 [00:00<?, ?it/s]

Creating training data for:  setahun_ID-1683_sentinel2_10m_med_mean_rev.tif


Processing clipping raster with vector:   0%|          | 0/145 [00:00<?, ?it/s]

Creating training data for:  setahun_ID-1793_sentinel2_10m_med_mean_rev.tif


Processing clipping raster with vector:   0%|          | 0/145 [00:00<?, ?it/s]

Finish


In [8]:
data_['TRAIN_CLASS'].unique()

array([ 7,  5,  9, 31,  2,  1,  4, 11, 33, 10,  6, 12])

## Encoding Landform Following the PODES Topography Level

In [9]:
dict_data_landform={0:'others',11:'others',12:'others',14:'others',15:'others',21: 'u-slope',22:'u-slope',
                   31:'l-slope',32:'l-slope',41:'valley',42:'valley',24:'flat',34:'flat',13:'others',
                    33:'l-slope',23:'u-slope'}
data_['PODES_landform']=data_['alos_landform'].apply(lambda y: dict_data_landform[y])

In [10]:
data_.columns

Index(['x', 'y', 'alos_landform', 'alos_slope', 'alos_dsm', 'wet_mean',
       'green_mean', 'bright_mean', 'ARVI_mean', 'SAVI_mean', 'NDBI_mean',
       'mNDWI_mean', 'NDWI_mean', 'mNDVI_mean', 'NDVI_mean', 'wet_p50',
       'green_p50', 'bright_p50', 'ARVI_p50', 'SAVI_p50', 'NDBI_p50',
       'mNDWI_p50', 'NDWI_p50', 'mNDVI_p50', 'NDVI_p50', 'S2_B12mean',
       'S2_B11mean', 'S2_B8mean', 'S2_B4mean', 'S2_B3mean', 'S2_B2mean',
       'S2_B12med', 'S2_B11med', 'S2_B8med', 'S2_B4med', 'S2_B3med',
       'S2_B2med', 'TRAIN_CLASS', 'PODES_landform'],
      dtype='object')

## Exporting data to Google Drive

In [11]:
file_name='kerangka-spasial/ml_learning/train_data/'+train_data
data_ready=data_[['x', 'y', 'PODES_landform', 'alos_slope', 'alos_dsm', 'wet_mean',
       'green_mean', 'bright_mean', 'ARVI_mean', 'SAVI_mean', 'NDBI_mean',
       'mNDWI_mean', 'NDWI_mean', 'mNDVI_mean', 'NDVI_mean', 'wet_p50',
       'green_p50', 'bright_p50', 'ARVI_p50', 'SAVI_p50', 'NDBI_p50',
       'mNDWI_p50', 'NDWI_p50', 'mNDVI_p50', 'NDVI_p50', 'S2_B12mean',
       'S2_B11mean', 'S2_B8mean', 'S2_B4mean', 'S2_B3mean', 'S2_B2mean',
       'S2_B12med', 'S2_B11med', 'S2_B8med', 'S2_B4med', 'S2_B3med',
       'S2_B2med', 'TRAIN_CLASS']]

data_ready.to_csv(file_name, index=False,sep=';')

In [12]:
data_ready.head()

,x,y,PODES_landform,alos_slope,alos_dsm,wet_mean,green_mean,bright_mean,ARVI_mean,SAVI_mean,NDBI_mean,mNDWI_mean,NDWI_mean,mNDVI_mean,NDVI_mean,wet_p50,green_p50,bright_p50,ARVI_p50,SAVI_p50,NDBI_p50,mNDWI_p50,NDWI_p50,mNDVI_p50,NDVI_p50,S2_B12mean,S2_B11mean,S2_B8mean,S2_B4mean,S2_B3mean,S2_B2mean,S2_B12med,S2_B11med,S2_B8med,S2_B4med,S2_B3med,S2_B2med,TRAIN_CLASS
0,107.690261,-6.634822,valley,5.40177,398.0,-0.094095,0.067978,0.419202,0.787719,0.405248,-0.260893,-0.265292,-0.486203,1.858596,0.605610,-0.091247,0.092720,0.400366,0.903238,0.438739,-0.275891,-0.287014,-0.532428,2.039340,0.670228,68.798027,96.839706,164.954666,42.261246,58.186501,62.809464,56.479702,139.555298,170.142776,31.422253,49.838886,52.759262,7
1,107.690081,-6.634912,valley,5.40177,398.0,-0.098851,0.040318,0.389891,0.791605,0.355214,-0.202230,-0.270012,-0.445274,-4.925207,0.574403,-0.095318,0.054284,0.374398,0.878398,0.362691,-0.199345,-0.285615,-0.479323,2.961947,0.628452,69.760895,96.667244,144.448334,42.318188,57.692631,63.338772,55.348598,136.571884,141.425430,30.314104,48.487396,53.279739,7
2,107.690171,-6.634912,valley,5.40177,398.0,-0.097156,0.048606,0.398074,0.795134,0.370626,-0.217392,-0.275533,-0.461315,14.396158,0.585361,-0.099691,0.065188,0.376194,0.895780,0.386196,-0.222863,-0.322560,-0.511234,2.797582,0.652223,68.798027,96.839706,150.316513,42.108616,57.485188,63.633144,56.479702,139.555298,148.563202,29.797562,48.696815,51.630825,7
3,107.690261,-6.634912,valley,5.40177,398.0,-0.097152,0.055714,0.402599,0.792344,0.382683,-0.230632,-0.278274,-0.473623,2.750324,0.594343,-0.097761,0.076209,0.382709,0.896874,0.406332,-0.247889,-0.325840,-0.530163,2.423103,0.666620,68.798027,96.839706,154.487564,41.684929,56.918079,62.559464,56.479702,139.555298,153.816849,30.122253,49.195324,52.055828,7
4,107.690351,-6.634912,valley,9.79179,401.0,-0.100206,0.064968,0.416922,0.786592,0.399995,-0.241810,-0.285554,-0.488161,2.528369,0.602909,-0.099101,0.084859,0.395676,0.893841,0.431173,-0.265474,-0.329254,-0.527031,2.116990,0.664421,70.295296,99.553268,162.645065,42.359089,57.373928,62.676620,58.960464,141.701599,163.593033,30.213974,47.966301,53.440495,7
